In [1]:
# Import libraries 
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

# Code obtained on https://www.kaggle.com/datatheque/association-rules-mining-market-basket-analysis

### Part 1: Data Preparation
#### A. Load order data

In [2]:
orders = pd.read_csv('InstacartChallenge/order_products_prior.csv')

#### B. Convert order data into format expected by the association rules function

In [3]:
# Convert from DataFrame to a Series, with order_id as index and item_id as value
orders = orders.set_index('order_id')['product_id'].rename('item_id')
orders.head(10)

order_id
2    33120
2    28985
2     9327
2    45918
2    30035
2    17794
2    40141
2     1819
2    43668
3    33754
Name: item_id, dtype: int64

#### C. Display summary statistics for order data

### Part 2: Association Rules Function

#### A. Helper functions to the main association rules function

In [4]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]

#### B. Association rules function

In [5]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
        
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

#### Part 3: Association Rules Mining

In [6]:
%%time
rules = association_rules(orders, 0.0001)

Starting order_item:               32434489
Items with support >= 0.0001:           48581
Remaining order_item:              32431787
Remaining orders with 2+ items:     3058096
Remaining order_item:              32275022


C:\Users\t514970\AppData\Local\Continuum\Miniconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


Item pairs:                        54073372
Item pairs with support >= 0.0001:    9997882

Wall time: 1h 22min 32s


In [8]:
# Replace item ID with item name and display association rules
item_name   = pd.read_csv('InstacartChallenge/products.csv')
item_name   = item_name.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
rules_final = merge_item_name(rules, item_name).sort_values('lift', ascending=False)
display(rules_final)

itemA  \
0       Organic Strawberry Chia Lowfat 2% Cottage Cheese   
1                    Grain Free Chicken Formula Cat Food   
3              Organic Fruit Yogurt Smoothie Mixed Berry   
9      Nonfat Strawberry With Fruit On The Bottom Gre...   
10        Organic Grapefruit Ginger Sparkling Yerba Mate   
11      Baby Food Pouch - Roasted Carrot Spinach & Beans   
12       Unsweetened Whole Milk Mixed Berry Greek Yogurt   
23                           Uncured Cracked Pepper Beef   
24                                  Organic Mango Yogurt   
2                    Grain Free Chicken Formula Cat Food   
25                               Raspberry Essence Water   
13              Unsweetened Whole Milk Strawberry Yogurt   
14             Unsweetened Whole Milk Peach Greek Yogurt   
44     Oh My Yog! Pacific Coast Strawberry Trilayer Y...   
55     Mighty 4 Kale, Strawberry, Amaranth & Greek Yo...   
20             Unsweetened Whole Milk Peach Greek Yogurt   
65              0% Greek, Blueberry on the Bottom Yogurt   
15       Unsweetened Whole Milk Mixed Berry Greek Yogurt   
43             Unsweetened Whole Milk Peach Greek Yogurt   
26                          Unsweetened Blackberry Water   
19       Unsweetened Whole Milk Mixed Berry Greek Yogurt   
16              Unsweetened Whole Milk Strawberry Yogurt   
56     Mighty 4 Sweet Potato, Blueberry, Millet & Gre...   
74                                Sweet Potatoes Stage 2   
79                                     Compostable Forks   
75                     Organic Stage 2 Carrots Baby Food   
42              Unsweetened Whole Milk Strawberry Yogurt   
21         Unsweetened Whole Milk Blueberry Greek Yogurt   
17         Unsweetened Whole Milk Blueberry Greek Yogurt   
83                            Cream Top Blueberry Yogurt   
...                                                  ...   
22444                                        Large Lemon   
2577                                           Red Onion   
250                              Roasted Pine Nut Hummus   
655                        Organic Large Green Asparagus   
40897                                             Banana   
2652                                             Spinach   
2722                                          Sour Cream   
11143                                Organic Blueberries   
2537                                        Green Onions   
1386                                 2% Reduced Fat Milk   
3291                                 2% Reduced Fat Milk   
530                               Chocolate Chip Cookies   
10681                                        Half & Half   
5446                         Organic Reduced Fat 2% Milk   
11455                                             Banana   
11421                             Bag of Organic Bananas   
2568                 Asparation/Broccolini/Baby Broccoli   
19596                                             Banana   
2319                                    Fridge Pack Cola   
11017                               Organic Baby Spinach   
22572                                Organic Raspberries   
11012                               Organic Strawberries   
246                                                 Soda   
11555                                             Banana   
1474                                        Strawberries   
7271                                Organic Strawberries   
6763                                Organic Hass Avocado   
4387                                     Organic Avocado   
2596                                              Banana   
670                               Bag of Organic Bananas   

                                                   itemB  freqAB  supportAB  \
0             Organic Cottage Cheese Blueberry Acai Chia     306   0.010155   
1                     Grain Free Turkey Formula Cat Food     318   0.010553   
3                  Apple Blueberry Fruit Yogurt Smoothie     349   0.011582   
9             

#### Part 4: Conclusion

From the output above, we see that the top associations are not surprising, with one flavor of an item being purchased with another flavor from the same item family (eg: Strawberry Chia Cottage Cheese with Blueberry Acai Cottage Cheese, Chicken Cat Food with Turkey Cat Food, etc). As mentioned, one common application of association rules mining is in the domain of recommender systems. Once item pairs have been identified as having positive relationship, recommendations can be made to customers in order to increase sales. And hopefully, along the way, also introduce customers to items they never would have tried before or even imagined existed!